# Curso de SQL Avanzado

<img src="https://amei.mx/wp-content/uploads/2016/08/Scotiabank-logo.jpg" alt="Scotiabank Logo" height="60">
<img src="https://www.belatrix.com/wp-content/uploads/2023/08/belatrix-logosweb-1.png" alt="Belatrix Logo" height="60">


**[Scotiabank](https://www.scotiabank.com.mx)** | **[Belatrix](https://www.belatrix.com)**

Instructor: [Alan Badillo Salas](alan@nomadacode.com)

## Casos de uso y escenarios prácticos

### Financiera Express

#### 1. Tablas de la Financiera Express

In [1]:
-- Creación de la tabla de Cliente
CREATE TABLE Cliente (
    ID_Cliente INT PRIMARY KEY,
    Nombre NVARCHAR(100),
    Apellido_Paterno NVARCHAR(100),
    Apellido_Materno NVARCHAR(100),
    Correo_Electronico NVARCHAR(255),
    Fecha_Registro DATETIME,
    Monto_Nomina DECIMAL(10, 2)
);

-- Creación de la tabla de Préstamos
CREATE TABLE Prestamos (
    ID_Prestamo INT PRIMARY KEY,
    ID_Cliente INT,
    Monto_Nomina DECIMAL(10, 2),
    Monto_Prestamo DECIMAL(10, 2),
    Monto_Comision DECIMAL(10, 2),
    Fecha_Solicitud DATETIME,
    Fecha_Aprobacion DATETIME,
    Activo BIT,
    FOREIGN KEY (ID_Cliente) REFERENCES Cliente(ID_Cliente)
);

-- Creación de la tabla de Abonos a Préstamos
CREATE TABLE Abonos_Prestamos (
    ID_Abono INT PRIMARY KEY,
    ID_Prestamo INT,
    ID_Cliente INT,
    Monto_Abono DECIMAL(10, 2),
    Fecha_Abono DATETIME,
    FOREIGN KEY (ID_Prestamo) REFERENCES Prestamos(ID_Prestamo),
    FOREIGN KEY (ID_Cliente) REFERENCES Cliente(ID_Cliente)
);

-- Creación de la tabla de Liquidaciones de Préstamos
CREATE TABLE Liquidaciones_Prestamos (
    ID_Liquidacion INT PRIMARY KEY,
    ID_Prestamo INT,
    ID_Cliente INT,
    ID_Abono_Ultimo INT,
    Numero_Abono INT,
    Fecha_Liquidacion DATETIME,
    Fecha_Confirmacion DATETIME,
    Fecha_Aceptacion DATETIME,
    FOREIGN KEY (ID_Prestamo) REFERENCES Prestamos(ID_Prestamo),
    FOREIGN KEY (ID_Cliente) REFERENCES Cliente(ID_Cliente),
    FOREIGN KEY (ID_Abono_Ultimo) REFERENCES Abonos_Prestamos(ID_Abono)
);

-- Creación de la tabla de Cobranza
CREATE TABLE Cobranza (
    ID_Cobranza INT PRIMARY KEY,
    ID_Prestamo INT,
    ID_Cliente INT,
    ID_Liquidacion INT,
    Fecha_Registro DATETIME,
    Fecha_Revision DATETIME,
    FOREIGN KEY (ID_Prestamo) REFERENCES Prestamos(ID_Prestamo),
    FOREIGN KEY (ID_Cliente) REFERENCES Cliente(ID_Cliente),
    FOREIGN KEY (ID_Liquidacion) REFERENCES Liquidaciones_Prestamos(ID_Liquidacion)
);


Commands completed successfully.

Total execution time: 00:00:00.085

In [ ]:
-- Eliminar las tablas en el orden correcto para evitar errores de claves foráneas
DROP TABLE IF EXISTS Cobranza;
DROP TABLE IF EXISTS Liquidaciones_Prestamos;
DROP TABLE IF EXISTS Abonos_Prestamos;
DROP TABLE IF EXISTS Prestamos;
DROP TABLE IF EXISTS Cliente;


#### 2. Procedimientos para la correcta inserción

In [6]:
CREATE PROCEDURE InsertarCliente
    @Nombre NVARCHAR(100),
    @Apellido_Paterno NVARCHAR(100),
    @Apellido_Materno NVARCHAR(100),
    @Correo_Electronico NVARCHAR(255),
    @Monto_Nomina DECIMAL(10, 2)
AS
BEGIN
    DECLARE @ID_Cliente INT = (SELECT ISNULL(MAX(ID_Cliente), 0) + 1 FROM Cliente);

    -- Insertar el nuevo cliente en la tabla Cliente
    INSERT INTO Cliente (ID_Cliente, Nombre, Apellido_Paterno, Apellido_Materno, Correo_Electronico, Fecha_Registro, Monto_Nomina)
    VALUES (@ID_Cliente, @Nombre, @Apellido_Paterno, @Apellido_Materno, @Correo_Electronico, SYSDATETIME(), @Monto_Nomina);
    
    -- Opcional: Devolver el ID del cliente insertado
    SELECT @ID_Cliente  AS ID_Cliente;
END;

Commands completed successfully.

Total execution time: 00:00:00.073

In [5]:
DROP PROCEDURE InsertarCliente;

Commands completed successfully.

Total execution time: 00:00:00.072

In [16]:
EXEC InsertarCliente
    @Nombre = 'Paty',
    @Apellido_Paterno = 'Pérez',
    @Apellido_Materno = 'López',
    @Correo_Electronico = 'paty.perez@ejemplo.com',
    @Monto_Nomina = 25000;

The statement has been terminated.

(1 row affected)

Total execution time: 00:00:00.070

(No column name)
2


: Msg 2601, Level 14, State 1, Procedure InsertarCliente, Line 12
Cannot insert duplicate key row in object 'dbo.Cliente' with unique index 'idx_nombre_apellidos'. The duplicate key value is (Paty, Pérez, López).

In [19]:
SELECT * FROM Cliente

(2 rows affected)

Total execution time: 00:00:00.072

ID_Cliente,Nombre,Apellido_Paterno,Apellido_Materno,Correo_Electronico,Fecha_Registro,Monto_Nomina
1,Paty,Pérez,López,paty.perez@ejemplo.com,2024-04-29 06:59:29.577,25000.00
2,Pedro,Pérez,López,pedro.perez@ejemplo.com,2024-04-29 07:00:11.800,25000.00


In [10]:
DELETE Cliente WHERE ID_Cliente = 3

(1 row affected)

Total execution time: 00:00:00.080

In [12]:
CREATE UNIQUE INDEX idx_nombre_apellidos ON Cliente (Nombre, Apellido_Paterno, Apellido_Materno)

Commands completed successfully.

Total execution time: 00:00:00.083

In [43]:
CREATE PROCEDURE InsertarPrestamo
    @ID_Cliente INT,
    @Monto_Prestamo DECIMAL(10, 2)
AS
BEGIN
    -- Verificar que el cliente no tenga un préstamo pendiente de aprobación
    IF EXISTS(SELECT TOP(1) ID_Prestamo FROM Prestamos WHERE ID_Cliente = @ID_Cliente AND Fecha_Aprobacion IS NULL)
    BEGIN
        RAISERROR('El cliente tiene un préstamo pendiente de aprobación.', 16, 1);
        RETURN;
    END

    -- Verificar que el cliente no tenga un préstamo activo
    IF EXISTS(SELECT TOP(1) ID_Prestamo FROM Prestamos WHERE ID_Cliente = @ID_Cliente AND Activo = 1)
    BEGIN
        RAISERROR('El cliente tiene un préstamo activo.', 16, 1);
        RETURN;
    END

    DECLARE @ID_Prestamo INT = (SELECT ISNULL(MAX(ID_Prestamo), 0) + 1 FROM Prestamos);

    DECLARE @Monto_Nomina DECIMAL(10, 2);
    DECLARE @Monto_Comision DECIMAL(10, 2);

    -- Obtener el monto de nómina actual del cliente
    SET @Monto_Nomina = (SELECT TOP(1) Monto_Nomina FROM Cliente WHERE ID_Cliente = @ID_Cliente);

    SET @Monto_Comision = (SELECT @Monto_Prestamo * 0.05);

    -- Verificar que el monto del préstamo no exceda el monto de la nómina
    IF @Monto_Prestamo + @Monto_Comision > @Monto_Nomina
    BEGIN
        RAISERROR('El monto del préstamo más comisión no puede exceder el monto de la nómina.', 16, 1);
        RETURN;
    END

    -- Insertar el nuevo préstamo en la tabla Prestamos
    INSERT INTO Prestamos (ID_Prestamo, ID_Cliente, Monto_Nomina, Monto_Prestamo, Monto_Comision, Fecha_Solicitud, Fecha_Aprobacion, Activo)
    VALUES (@ID_Prestamo, @ID_Cliente, @Monto_Nomina, @Monto_Prestamo, @Monto_Comision, SYSDATETIME(), NULL, 0);
    
    -- Opcional: Devolver el ID del préstamo insertado
    SELECT @ID_Prestamo AS ID_Prestamo
END;


Commands completed successfully.

Total execution time: 00:00:00.082

In [42]:
DROP PROCEDURE InsertarPrestamo

Commands completed successfully.

Total execution time: 00:00:00.071

In [72]:
EXEC InsertarPrestamo
    @ID_Cliente=1,
    @Monto_Prestamo=11000;

: Msg 50000, Level 16, State 1, Procedure InsertarPrestamo, Line 16
El cliente tiene un préstamo activo.

Total execution time: 00:00:00.078

In [21]:
CREATE UNIQUE INDEX idx_prestamos_id_cliente ON Prestamos(ID_Cliente)

Commands completed successfully.

Total execution time: 00:00:00.081

In [37]:
DROP INDEX idx_prestamos_id_cliente ON Prestamos;

Commands completed successfully.

Total execution time: 00:00:00.072

In [35]:
SELECT * FROM Prestamos

(1 row affected)

Total execution time: 00:00:00.072

ID_Prestamo,ID_Cliente,Monto_Nomina,Monto_Prestamo,Monto_Comision,Fecha_Solicitud,Fecha_Aprobacion,Activo
1,1,25000.00,10000.00,500.00,2024-04-29 07:16:29.047,NULL,0


In [32]:
DELETE FROM Prestamos

(1 row affected)

Total execution time: 00:00:00.070

In [65]:
CREATE PROCEDURE RechazarPrestamo
    @ID_Prestamo INT
AS
BEGIN
    -- Validar que el préstamo no esté aprobado
    IF EXISTS(SELECT TOP(1) ID_Prestamo FROM Prestamos WHERE ID_Prestamo = @ID_Prestamo AND Fecha_Aprobacion IS NOT NULL)
    BEGIN
        RAISERROR('El préstamo ya está aprobado o rechazado.', 16, 1);
        RETURN;
    END

    UPDATE Prestamos SET Fecha_Aprobacion = SYSDATETIME(), Activo = 1 WHERE ID_Prestamo = @ID_Prestamo;
END

Commands completed successfully.

Total execution time: 00:00:00.072

In [64]:
DROP PROCEDURE RechazarPrestamo

Commands completed successfully.

Total execution time: 00:00:00.072

In [52]:
EXEC RechazarPrestamo
    @ID_Prestamo=1;

: Msg 50000, Level 16, State 1, Procedure RechazarPrestamo, Line 8
El préstamo ya está aprobado.

Total execution time: 00:00:00.071

In [73]:
SELECT * FROM Prestamos

(2 rows affected)

Total execution time: 00:00:00.074

ID_Prestamo,ID_Cliente,Monto_Nomina,Monto_Prestamo,Monto_Comision,Fecha_Solicitud,Fecha_Aprobacion,Activo
1,1,25000.00,10000.00,500.00,2024-04-29 07:16:29.047,2024-04-29 07:41:40.917,0
2,1,25000.00,11000.00,550.00,2024-04-29 07:37:12.520,2024-04-29 07:41:59.190,1


In [63]:
CREATE PROCEDURE AprobarPrestamo
    @ID_Prestamo INT
AS
BEGIN
    -- Validar que el préstamo no esté aprobado
    IF EXISTS(SELECT TOP(1) ID_Prestamo FROM Prestamos WHERE ID_Prestamo = @ID_Prestamo AND Fecha_Aprobacion IS NOT NULL)
    BEGIN
        RAISERROR('El préstamo ya está aprobado o rechazado.', 16, 1);
        RETURN;
    END


    UPDATE Prestamos SET Fecha_Aprobacion = SYSDATETIME(), Activo = 1 WHERE ID_Prestamo = @ID_Prestamo;
END

Commands completed successfully.

Total execution time: 00:00:00.072

In [62]:
DROP PROCEDURE AprobarPrestamo

Commands completed successfully.

Total execution time: 00:00:00.071

In [70]:
EXEC AprobarPrestamo
    @ID_Prestamo=2;

: Msg 50000, Level 16, State 1, Procedure AprobarPrestamo, Line 8
El préstamo ya está aprobado o rechazado.

Total execution time: 00:00:00.072

In [71]:
SELECT * FROM Prestamos

(2 rows affected)

Total execution time: 00:00:00.073

ID_Prestamo,ID_Cliente,Monto_Nomina,Monto_Prestamo,Monto_Comision,Fecha_Solicitud,Fecha_Aprobacion,Activo
1,1,25000.00,10000.00,500.00,2024-04-29 07:16:29.047,2024-04-29 07:41:40.917,0
2,1,25000.00,11000.00,550.00,2024-04-29 07:37:12.520,2024-04-29 07:41:59.190,1
